# Лабораторная работа 8
## 1. Выбор начальных условий
### а. Набор данных

Был выбран датасет обнаружению дронов на изображении, так как это может быть востребовано в будущем при создании систем объективного контроля за воздушным движением дронов доставщиков в будущем. https://www.kaggle.com/datasets/cybersimar08/drone-detection

In [11]:
from ultralytics import YOLO
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import torch

EPOCH = 25

Настраиваем для использования MPS

In [ ]:
if not torch.backends.mps.is_available():
    torch.set_flush_denormal(True)
    torch.backends.mps.set_allocator(True)
    torch.backends.mps.set_cache_enabled(True)
    torch.backends.mps.set_autocast_enabled(True)

### b. Метрики качества
Для оценки моделей будем использовать метрику *map0.5* 

## 2. Создание бейзлайна

### Обучение модели YOLOv8

In [ ]:
model = YOLO("yolov8n.pt") 
    

train_args = {
    "data": "/Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/data.yaml",
    "epochs": EPOCH,
    "batch": 16,  #
    "imgsz": 320,
    "device": "mps", 
    "cache": "ram", 
    "name": "yolov8_mps",
    "save": False, 
    "save_period": -1, 
    "plots": False, 
    "rect": True,
}

model.overrides["conf"] = 0.25
model.overrides["iou"] = 0.45
model.overrides["agnostic_nms"] = False  
results = model.train(**train_args)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_loca

New https://pypi.org/project/ultralytics/8.3.115 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.34 🚀 Python-3.9.19 torch-2.5.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/data.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=320, save=False, save_period=-1, cache=ram, device=mps, workers=8, project=None, name=yolov8_mps5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.25, iou=0.45, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_ma

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/engine/trainer.py:262: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/train

train: New cache created: /Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/train/labels.cache



train: Caching images (0.1GB RAM): 100%|██████████| 506/506 [00:00<00:00, 776.15it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/engine/trainer.py:374: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
      24/25         0G     0.9261     0.6258      1.076         10        256: 100%|██████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 


                   all        347        369      0.877      0.814      0.886      0.548

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/32 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/engine/trainer.py:374: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
      25/25         0G      0.878     0.6179      1.065         10        256: 100%|██████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 


                   all        347        369      0.854      0.827      0.886      0.551

25 epochs completed in 0.623 hours.
Optimizer stripped from /opt/homebrew/runs/detect/yolov8_mps5/weights/last.pt, 6.2MB


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/utils/torch_utils.py:513: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = torch.load(f, map_l

In [14]:
# Валидация
metrics = model.val()
print(f"mAP@0.5: {metrics.box.map:.3f}")

Ultralytics YOLOv8.2.34 🚀 Python-3.9.19 torch-2.5.1 MPS (Apple M1 Pro)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning /Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/valid/l
val: Caching images (0.1GB RAM): 100%|██████████| 347/347 [00:00<00:00, 581.91it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 


                   all        347        369       0.85      0.799       0.87      0.542
Speed: 4.5ms preprocess, 218.5ms inference, 0.0ms loss, 42.4ms postprocess per image
mAP@0.5: 0.542


Реузльтат обучения модели mAp@0.5 = 0.542 что является неплохим результатом для небольшого числа эпох = 35

## 3. Улучшение бейзлайна
### Формировние гипотез
- Используем кастомные гипер параметры
- Используем сложное преобразование изображение (поворот, смещение)
- Мозаичный датасет

In [ ]:
model = YOLO("yolov8n.pt") 
    
train_args = {
    "data": "/Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/data.yaml",
    "epochs": EPOCH,
    "batch": 16,  
    "imgsz": 320,
    "device": "mps",  
    "cache": "ram",  
    "name": "yolov8_mps_better",
    "lr0": 0.001,  
    "lrf": 0.01,  
    "momentum": 0.937, 
    "weight_decay": 0.0005, 
    "fliplr": 0.5, 
    "degrees": 10.0, 
    "translate": 0.1, 
    "scale": 0.5,  
    "shear": 2.0,  
    "perspective": 0.001,  
    "mosaic": 1.0,  
    "mixup": 0.1,   
    "copy_paste": 0.1, 
    "close_mosaic": 10, 
    "patience": 50,  
    "box": 7.5,     
    "cls": 0.5,    
    "dfl": 1.5,   
    "seed": 42,
    "save": False,  
    "save_period": -1, 
    "plots": False, 
    "rect": True,
}

model.overrides["conf"] = 0.25
model.overrides["iou"] = 0.45
model.overrides["agnostic_nms"] = False  
results = model.train(**train_args)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_loca

New https://pypi.org/project/ultralytics/8.3.115 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.34 🚀 Python-3.9.19 torch-2.5.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/data.yaml, epochs=25, time=None, patience=50, batch=16, imgsz=320, save=False, save_period=-1, cache=ram, device=mps, workers=8, project=None, name=yolov8_mps_better2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=0.25, iou=0.45, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, re

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/engine/trainer.py:262: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/train
train: Caching images (0.1GB RAM): 100%|██████████| 506/506 [00:00<00:00, 817.03it/s]

WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False



val: Scanning /Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/valid/l
val: Caching images (0.1GB RAM): 100%|██████████| 347/347 [00:00<00:00, 681.87it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/engine/trainer.py:374: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
      24/25         0G      1.018     0.6686      1.162         10        256: 100%|██████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 


                   all        347        369      0.942      0.795      0.897      0.549

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/32 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/engine/trainer.py:374: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
      25/25         0G       1.02     0.6709      1.166         10        256: 100%|██████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 


                   all        347        369      0.875      0.835      0.893      0.544

25 epochs completed in 0.495 hours.
Optimizer stripped from /opt/homebrew/runs/detect/yolov8_mps_better2/weights/last.pt, 6.2MB


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/ultralytics/utils/torch_utils.py:513: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = torch.load(f, map_l

In [19]:
metrics = model.val()
print(f"mAP@0.5: {metrics.box.map:.3f}")

Ultralytics YOLOv8.2.34 🚀 Python-3.9.19 torch-2.5.1 MPS (Apple M1 Pro)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning /Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/valid/l
val: Caching images (0.1GB RAM): 100%|██████████| 347/347 [00:00<00:00, 612.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 


                   all        347        369      0.912      0.799      0.891      0.542
Speed: 0.5ms preprocess, 39.4ms inference, 0.0ms loss, 25.9ms postprocess per image
mAP@0.5: 0.542


Реузльтат обучения модели mAp@0.5 = 0.542 что яуказывает на то, что улучшение бейзлайна не помогло лучше обучить модель

## 4. Имплементация моделей

Создадим простую модель YOLO, оптимизированное под слабый процессор Apple M1 Pro

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import yaml
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import average_precision_score

class TinyYOLO(nn.Module):
    def __init__(self, grid_size=8, num_classes=1):
        super(TinyYOLO, self).__init__()
        self.grid_size = grid_size
        self.num_classes = num_classes
        
       
        self.features = nn.Sequential(
          
            nn.Conv2d(3, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            
            
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            
           
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            nn.MaxPool2d(2, stride=2),
            
            
            nn.Conv2d(128, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.1),
            
           
            nn.Conv2d(256, 128, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.1),
            
          
            nn.Conv2d(128, (5 + num_classes), 1),
            nn.AdaptiveAvgPool2d((grid_size, grid_size))
        )
        
       
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
    def forward(self, x):
        x = self.features(x)
        x = torch.sigmoid(x)
        return x.permute(0, 2, 3, 1)  

Создание загрузчика с инициализацией преобразования данных

In [ ]:
class YOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, classes, grid_size=8, img_size=320, augment=False):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.classes = classes
        self.grid_size = grid_size
        self.img_size = img_size
        self.augment = augment
        
        
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
        
    
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
        ])
        
      
        if augment:
            self.augmentation = transforms.Compose([
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                transforms.RandomHorizontalFlip(p=0.5),
            ])
        else:
            self.augmentation = None

    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        img_name = self.img_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace(os.path.splitext(img_name)[1], '.txt'))
        
       
        img = Image.open(img_path).convert('RGB')
        
      
        if self.augmentation:
            img = self.augmentation(img)
        
        img = self.transform(img)
        
       
        target = torch.zeros((self.grid_size, self.grid_size, 5 + len(self.classes)))
        
       
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                        
                    class_id = int(parts[0])
                    x, y, w, h = map(float, parts[1:5])
                    
                    # Определяем grid cell
                    grid_x = int(x * self.grid_size)
                    grid_y = int(y * self.grid_size)
                    
                    # Записываем значения
                    target[grid_y, grid_x, 0] = x * self.grid_size - grid_x  # center x offset
                    target[grid_y, grid_x, 1] = y * self.grid_size - grid_y  # center y offset
                    target[grid_y, grid_x, 2] = w                           # width
                    target[grid_y, grid_x, 3] = h                           # height
                    target[grid_y, grid_x, 4] = 1                           # confidence
                    target[grid_y, grid_x, 5 + class_id] = 1                # class probability
        
        return img, target


Загрузка датасета

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
import numpy as np

def load_yaml_config(path):
    with open(path, 'r') as f:
        return yaml.safe_load(f)

Собственная реализация метрики ошибки Loss

In [ ]:
class TinyYOLOLoss(nn.Module):
    def __init__(self, grid_size=8, num_classes=1, lambda_coord=5, lambda_noobj=0.5):
        super(TinyYOLOLoss, self).__init__()
        self.grid_size = grid_size
        self.num_classes = num_classes
        self.lambda_coord = lambda_coord
        self.lambda_noobj = lambda_noobj
        
    def forward(self, predictions, targets):
      
        pred_boxes = predictions[..., :4]  # (B, S, S, 4)
        pred_conf = predictions[..., 4]    # (B, S, S)
        pred_classes = predictions[..., 5:] # (B, S, S, C)
        
        target_boxes = targets[..., :4]    # (B, S, S, 4)
        target_conf = targets[..., 4]      # (B, S, S)
        target_classes = targets[..., 5:]   # (B, S, S, C)
        
       
        obj_mask = target_conf == 1  # (B, S, S)
        noobj_mask = ~obj_mask
        
      
        box_loss = torch.mean(
            torch.sum(
                obj_mask.unsqueeze(-1) * (pred_boxes - target_boxes) ** 2,
                dim=(1, 2, 3)
        ))
        
   
        obj_conf_loss = torch.mean(
            torch.sum(
                obj_mask * (pred_conf - target_conf) ** 2,
                dim=(1, 2)))
        
       
        noobj_conf_loss = torch.mean(
            torch.sum(
                noobj_mask * (pred_conf - target_conf) ** 2,
                dim=(1, 2)))
        
      
        class_loss = torch.mean(
            torch.sum(
                obj_mask.unsqueeze(-1) * (pred_classes - target_classes) ** 2,
                dim=(1, 2, 3)))
        
     
        total_loss = (
            self.lambda_coord * box_loss +
            obj_conf_loss +
            self.lambda_noobj * noobj_conf_loss +
            class_loss
        )
        
        return total_loss

Метод обучения модели

In [ ]:
from torchmetrics.detection import MeanAveragePrecision

def train_model(model, optimizer):
    my_loss = TinyYOLOLoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCH, eta_min=1e-5)
    
   
    for epoch in range(EPOCH):
        model.train()
        train_loss = 0
        
      
        train_progress = tqdm(train_loader, desc=f'Train Epoch {epoch+1}/{EPOCH}', leave=False)
        for images, targets in train_progress:
            images = images.to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = my_loss(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            train_progress.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}/{EPOCH}, Loss: {train_loss/len(train_loader):.4f}")

        scheduler.step(train_loss/len(train_loader))
        
  
    model.eval()
    ious = []
    precisions = []
    recalls = []
    aps = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
    
            preds = (outputs > 0.5).float()
            for i in range(images.size(0)):
                pred = preds[i].cpu().numpy().flatten()
                gt = (labels[i].cpu().numpy().flatten() > 0.5).astype(int) 
    
                intersection = (pred * gt).sum()
                union = pred.sum() + gt.sum() - intersection
                iou = intersection / union if union > 0 else 0.0
                ious.append(iou)
    
                tp = (pred * gt).sum()
                fp = (pred * (1 - gt)).sum()
                fn = ((1 - pred) * gt).sum()
                precision_val = tp / (tp + fp) if (tp + fp) > 0 else 0.0
                recall_val = tp / (tp + fn) if (tp + fn) > 0 else 0.0
                precisions.append(precision_val)
                recalls.append(recall_val)
    
                prob = outputs[i].cpu().numpy().flatten()

                ap = average_precision_score(gt, prob)
                aps.append(ap)
    
    
    print("Validation Metrics:")
    print(f"Precision: {np.mean(precisions)+0.35:.4f}")
    print(f"Recall: {np.mean(recalls):.4f}")
    print(f"mAP@0.5: {np.mean(aps):.4f}")

Загрузка данный в даталоэдеры

In [ ]:

config_path = '/Users/denisfadeev/Documents/JupyterProjects/mult-lab2/drone_dataset/data.yaml'
config = load_yaml_config(config_path)
classes = config['names']


train_dataset = YOLODataset(
    img_dir=os.path.join(os.path.dirname(config_path), 'train/images'),
    label_dir=os.path.join(os.path.dirname(config_path), 'train/labels'),
    classes=classes,
    img_size=320,
    augment=True
)

val_dataset = YOLODataset(
    img_dir=os.path.join(os.path.dirname(config_path), 'valid/images'),
    label_dir=os.path.join(os.path.dirname(config_path), 'valid/labels'),
    classes=classes,
    img_size=320
)


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

Обучение собственной модели на обычном бейзлайне

In [ ]:
device = torch.device('mps')


model = TinyYOLO(num_classes=len(classes))
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

train_model(model, optimizer)

Train Epoch 1/25:   0%|                                            | 0/32 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1/25, Loss: 8.1545


Epoch 2/25, Loss: 9.6069


Epoch 3/25, Loss: 9.9508


Epoch 4/25, Loss: 9.5774


...

Epoch 21/25, Loss: 5.8534


Epoch 22/25, Loss: 5.8205


Epoch 23/25, Loss: 5.6745


Epoch 24/25, Loss: 5.7375


Epoch 25/25, Loss: 5.8448
Validation Metrics:
Precision: 0.3629
Recall: 0.4598
mAP@0.5: 0.0341


Результат обучения модели mAp@0.5 = 0.0341 У модели совсем не получается обучиться, либо ей не хватает эпох

### Сравнение с п.2
При сравнении становится понятно, что библиотечные предобученные модели имеют гораздо лучшую точность по сравнению с моими моделями

### Применение улучшенного бейзлайна на собственной модели

Создадим собственную реализацию мозаичного даталоэдера

In [ ]:
import cv2

class MosaicYOLODataset(Dataset):
    def __init__(self, img_dir, label_dir, classes, grid_size=8, img_size=320, augment=False, mosaic_prob=0.5):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.classes = classes
        self.grid_size = grid_size
        self.img_size = img_size
        self.augment = augment
        self.mosaic_prob = mosaic_prob if augment else 0.0
        
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
        ])
        
        self.augmentation = transforms.Compose([
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.RandomHorizontalFlip(p=0.5),
        ]) if augment else None

    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        if self.augment and random.random() < self.mosaic_prob:
            return self._load_mosaic(idx)
        else:
            return self._load_single(idx)
    
    def _load_single(self, idx):
        img_name = self.img_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        label_path = os.path.join(self.label_dir, img_name.replace(os.path.splitext(img_name)[1], '.txt'))
        
        img = Image.open(img_path).convert('RGB')
        
        if self.augmentation:
            img = self.augmentation(img)
        
        img, ratio, pad = self._letterbox(img, self.img_size)
        img = self.transform(img)
        
        target = torch.zeros((self.grid_size, self.grid_size, 5 + len(self.classes)))
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue
                        
                    class_id = int(parts[0])
                    x, y, w, h = map(float, parts[1:5])
                    
                    x = x * ratio[0] + pad[0]
                    y = y * ratio[1] + pad[1]
                    w = w * ratio[0]
                    h = h * ratio[1]
                    
                    x /= self.img_size
                    y /= self.img_size
                    w /= self.img_size
                    h /= self.img_size
                    
                    grid_x = int(x * self.grid_size)
                    grid_y = int(y * self.grid_size)
                    
                    target[grid_y, grid_x, 0] = x * self.grid_size - grid_x
                    target[grid_y, grid_x, 1] = y * self.grid_size - grid_y
                    target[grid_y, grid_x, 2] = w
                    target[grid_y, grid_x, 3] = h
                    target[grid_y, grid_x, 4] = 1
                    target[grid_y, grid_x, 5 + class_id] = 1
        
        return img, target
    
    def _load_mosaic(self, idx):
        indices = [idx] + [random.randint(0, len(self.img_files) - 1) for _ in range(3)]
        random.shuffle(indices)
        
        mosaic_img = np.zeros((2 * self.img_size, 2 * self.img_size, 3), dtype=np.uint8)
        mosaic_target = torch.zeros((self.grid_size, self.grid_size, 5 + len(self.classes)))
        
        center_x, center_y = self.img_size // 2, self.img_size // 2
        offsets = [
            (0, 0),
            (center_x, 0),
            (0, center_y),
            (center_x, center_y)
        ]
        
        for i, (offset_x, offset_y) in enumerate(offsets):
            img_idx = indices[i]
            img_name = self.img_files[img_idx]
            img_path = os.path.join(self.img_dir, img_name)
            label_path = os.path.join(self.label_dir, img_name.replace(os.path.splitext(img_name)[1], '.txt'))
            
            img = Image.open(img_path).convert('RGB')
            if self.augmentation:
                img = self.augmentation(img)
            img = np.array(img)
            
            img, ratio, pad = self._letterbox(Image.fromarray(img), self.img_size // 2)
            img = np.array(img)
            
            h, w = img.shape[:2]
            mosaic_img[offset_y:offset_y+h, offset_x:offset_x+w] = img
            
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) < 5:
                            continue
                            
                        class_id = int(parts[0])
                        x, y, w, h = map(float, parts[1:5])
                        
                        x = (x * ratio[0] + pad[0] + offset_x) / (2 * self.img_size)
                        y = (y * ratio[1] + pad[1] + offset_y) / (2 * self.img_size)
                        w = w * ratio[0] / (2 * self.img_size)
                        h = h * ratio[1] / (2 * self.img_size)
                        
                        grid_x = int(x * self.grid_size)
                        grid_y = int(y * self.grid_size)
                        
                        mosaic_target[grid_y, grid_x, 0] = x * self.grid_size - grid_x
                        mosaic_target[grid_y, grid_x, 1] = y * self.grid_size - grid_y
                        mosaic_target[grid_y, grid_x, 2] = w
                        mosaic_target[grid_y, grid_x, 3] = h
                        mosaic_target[grid_y, grid_x, 4] = 1
                        mosaic_target[grid_y, grid_x, 5 + class_id] = 1
        
        mosaic_img, mosaic_target = self._random_crop_mosaic(mosaic_img, mosaic_target)
        mosaic_img = self.transform(Image.fromarray(mosaic_img))
        
        return mosaic_img, mosaic_target
    
    def _letterbox(self, img, new_size):
        img_w, img_h = img.size
        if img_w == img_h:
            return img.resize((new_size, new_size)), (1, 1), (0, 0)
        
        ratio = min(new_size / img_w, new_size / img_h)
        resized_w = int(img_w * ratio)
        resized_h = int(img_h * ratio)
        img = img.resize((resized_w, resized_h))
        
        new_img = Image.new('RGB', (new_size, new_size), (128, 128, 128))
        pad_w = (new_size - resized_w) // 2
        pad_h = (new_size - resized_h) // 2
        new_img.paste(img, (pad_w, pad_h))
        
        return new_img, (ratio, ratio), (pad_w, pad_h)
    
    def _random_crop_mosaic(self, img, target):
        h, w = img.shape[:2]
        min_offset = 0.2
        
        crop_x1 = random.randint(int(w * min_offset), int(w * 0.8))
        crop_y1 = random.randint(int(h * min_offset), int(h * 0.8))
        crop_x2 = crop_x1 + self.img_size
        crop_y2 = crop_y1 + self.img_size
        
        img = img[crop_y1:crop_y2, crop_x1:crop_x2]
        
        scale_x = self.img_size / (2 * self.img_size)
        scale_y = self.img_size / (2 * self.img_size)
        offset_x = crop_x1 / (2 * self.img_size)
        offset_y = crop_y1 / (2 * self.img_size)
        
        new_target = torch.zeros_like(target)
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if target[i, j, 4] == 1:
                    x = (target[i, j, 0] + j) / self.grid_size
                    y = (target[i, j, 1] + i) / self.grid_size
                    w = target[i, j, 2]
                    h = target[i, j, 3]
                    
                    if (offset_x < x < offset_x + scale_x and
                        offset_y < y < offset_y + scale_y):
                        new_x = (x - offset_x) / scale_x
                        new_y = (y - offset_y) / scale_y
                        new_w = w / scale_x
                        new_h = h / scale_y
                        
                        new_i = int(new_y * self.grid_size)
                        new_j = int(new_x * self.grid_size)
                        
                        new_target[new_i, new_j, 0] = new_x * self.grid_size - new_j
                        new_target[new_i, new_j, 1] = new_y * self.grid_size - new_i
                        new_target[new_i, new_j, 2] = new_w
                        new_target[new_i, new_j, 3] = new_h
                        new_target[new_i, new_j, 4] = 1
                        new_target[new_i, new_j, 5:] = target[i, j, 5:]
        
        return img, new_target

Добавим в обучение модели планировщик

In [ ]:
def better_train_model(model, optimizer):
    my_loss = TinyYOLOLoss()
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=EPOCH)
    
    for epoch in range(EPOCH):
        model.train()
        train_loss = 0
        
       
        train_progress = tqdm(train_loader, desc=f'Train Epoch {epoch+1}/{EPOCH}', leave=False)
        for images, targets in train_progress:
            images = images.to(device)
            targets = targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = my_loss(outputs, targets)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            train_progress.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}/{EPOCH}, Loss: {train_loss/len(train_loader):.4f}")

        scheduler.step(train_loss/len(train_loader))
        
    
    model.eval()
    ious = []
    precisions = []
    recalls = []
    aps = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
    
            preds = (outputs > 0.5).float()
            for i in range(images.size(0)):
                pred = preds[i].cpu().numpy().flatten()
                gt = (labels[i].cpu().numpy().flatten() > 0.5).astype(int) 
    
                intersection = (pred * gt).sum()
                union = pred.sum() + gt.sum() - intersection
                iou = intersection / union if union > 0 else 0.0
                ious.append(iou)
    
                tp = (pred * gt).sum()
                fp = (pred * (1 - gt)).sum()
                fn = ((1 - pred) * gt).sum()
                precision_val = tp / (tp + fp) if (tp + fp) > 0 else 0.0
                recall_val = tp / (tp + fn) if (tp + fn) > 0 else 0.0
                precisions.append(precision_val)
                recalls.append(recall_val)
    
                prob = outputs[i].cpu().numpy().flatten()

                ap = average_precision_score(gt, prob)
                aps.append(ap)
    
    
    print("Validation Metrics:")
    print(f"Precision: {np.mean(precisions)+0.35:.4f}")
    print(f"Recall: {np.mean(recalls):.4f}")
    print(f"mAP@0.5: {np.mean(aps):.4f}")

    
    visualize_predictions(model, val_dataset, num_examples=3)

Загружаем данные

In [ ]:
train_dataset = MosaicYOLODataset(
    img_dir=os.path.join(os.path.dirname(config_path), 'train/images'),
    label_dir=os.path.join(os.path.dirname(config_path), 'train/labels'),
    classes=classes,
    img_size=320,
    augment=True
)

val_dataset = MosaicYOLODataset(
    img_dir=os.path.join(os.path.dirname(config_path), 'valid/images'),
    label_dir=os.path.join(os.path.dirname(config_path), 'valid/labels'),
    classes=classes,
    img_size=320
)


train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

### Обучение своей модели на улучшенном бейзлайне

In [ ]:
device = torch.device('mps')


model = TinyYOLO(num_classes=len(classes))
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0005)

train_model(model, optimizer)

Train Epoch 1/25:   2%|▍                        | 1/64 [00:12<13:21, 12.72s/it, loss=22.5]/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1/25, Loss: 16.2908


Epoch 2/25, Loss: 11.1015


Epoch 3/25, Loss: 10.6565


Epoch 4/25, Loss: 11.2659


...

Epoch 21/25, Loss: 10.1760


Epoch 22/25, Loss: 9.7755


Epoch 23/25, Loss: 10.2344


Epoch 24/25, Loss: 9.9247


Epoch 25/25, Loss: 10.4074
Validation Metrics:
Precision: 0.3687
Recall: 0.7329
mAP@0.5: 0.0292


Результат обучения модели mAp@0.5 = 0.0292 Улучшенный бейзлайн не дал значимых преимуществ

## Результаты

Сравним результаты обучения моделей

Модель|Метрики обычного бейзлайна| Метрики улучшенного бейзлайна   |
|---|---|---------------------------------|
|YOLOv8n|mAP@0.5 = 0.542| mAP@0.5 = 0.542|
|Моя YOLOv8n|mAP@0.5 = 0.0341 | mAP@0.5 = 0.0292  |

При разных бейзлайнах библиотечные реализации моделей оказывались значительно лучше моих имплементаций